In [1]:
import sys
sys.path.append('..')
from theano import function, config, shared, sandbox
import theano.tensor as T
import numpy
import time
import theano_lstm
import numpy as np

Using gpu device 0: GeForce GTX TITAN X


In [3]:
reload(theano_lstm)
config.mode="FAST_RUN"
hidden_size = 40
network_opt = theano_lstm.lstm_rnn(300,
                  [(hidden_size, hidden_size)],
                  10, log_dir='test_log', dropout=0.2)
# network_opt.initialize_training_adadelta()

num_examples = 9
input_seq = np.random.normal(size=(2200, 300, num_examples)).astype('float32')
seq_lengths = [np.random.randint(low=500,high=2200) for x in range(num_examples)]

network_opt.generate_masks()

np.all(network_opt.process(input_seq, seq_lengths) == network_opt.process_unmodified_weights(input_seq, seq_lengths))

False

In [ ]:
network_opt.LSTM_stack.list_params()[3].get_value()

In [ ]:
num_examples = 9
input_seq = np.random.normal(size=(2200, 300, num_examples)).astype('float32')
seq_lengths = [np.random.randint(low=500,high=2200) for x in range(num_examples)]
answers = np.zeros(shape=(50, num_examples)).astype('float32')
answers[10, :] = 1

In [ ]:
network_opt.process(input_seq, seq_lengths)